<a href="https://colab.research.google.com/github/vir-dom/deep_learning_hw3/blob/main/deep_learning_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deep Learning homework 3

This is the third homework for the "Deep Learning a gyakorlatban..." course. My name is Virsinger Dominika (RSABSD). In this notebook I am trying to explain everything regarding my codes, it is going to be in these Text boxes.

For this exercise, I used the last 3 years' data (2019.11.01.-2022.11.14.) from http://idojarasbudapest.hu/archivalt-idojaras
<p> I collected the mentioned data into an Excel file (traindata.xlsx) which can be found attached. (To do this I used https://www.convertcsv.com/newsite/html-table-to-csv.htm)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import mean_squared_error
import datetime
from datetime import timedelta
from datetime import date
import calendar
random_state = 23

**Data preparation**

In [ ]:
data = pd.read_excel("./traindata.xlsx")
data

,Nap,Tmax\n [°C],Tmin\n [°C],Szél\n \t[m/s],Csapadék\n \t[mm]
0,2019.11.01\npéntek,9.7,5.3,3.5,0.0
1,2019.11.02\nszombat,11.1,6.3,4.8,0.0
2,2019.11.03\nvasárnap,16.4,9.8,6.6,0.4
3,2019.11.04\nhétfő,15.7,9.9,5.3,9.6
4,2019.11.05\nkedd,16.3,9.2,6.6,12.3
...,...,...,...,...,...
1105,2022.11.10\ncsütörtök,14.2,10.5,4.1,0.0
1106,2022.11.11\npéntek,13.5,9.3,1.9,0.0
1107,2022.11.12\nszombat,13.9,9.4,1.6,0.0
1108,2022.11.13\nvasárnap,13.1,8.3,1.8,0.0


It is quite obvious that this form isn't really usable, so we have to make a few changes:
<li>I am adding an id to each row
<li>I am adding a month variable that I'll later use as a predictor variable
<li>I remove the "szél" and "csapadék" attributes, because I only want to use Tmin and Tmax at modelling
<li>I transform the date, bacause it is quite strange now

In [ ]:
id_list = []
date_list = []
month_list = []
Tmin_list = []
Tmax_list = []
counter = 0
for i, row in data.iterrows():
    id_list.append(counter)
    counter += 1
    date_list.append(row['Nap'][0:10])
    month_list.append(row['Nap'][5:7])
    Tmin_list.append(row['Tmin\n [°C]'])
    Tmax_list.append(row['Tmax\n [°C]'])
weather = pd.DataFrame(data={'id':id_list,'date':date_list,'month':month_list,
                             'Tmin':Tmin_list, 'Tmax':Tmax_list})

In [ ]:
weather.head()

,id,date,month,Tmin,Tmax
0,0,2019.11.01,11,5.3,9.7
1,1,2019.11.02,11,6.3,11.1
2,2,2019.11.03,11,9.8,16.4
3,3,2019.11.04,11,9.9,15.7
4,4,2019.11.05,11,9.2,16.3


We need to change the type of 'date' attribute to datetime.

In [ ]:
weather['date']= pd.to_datetime(weather['date'])

In [ ]:
weather.head()

,id,date,month,Tmin,Tmax
0,0,2019-11-01,11,5.3,9.7
1,1,2019-11-02,11,6.3,11.1
2,2,2019-11-03,11,9.8,16.4
3,3,2019-11-04,11,9.9,15.7
4,4,2019-11-05,11,9.2,16.3


Since the data is dependent on the date, we cannot easily do a train-test-split. We have to make predictor variables:
<p>Each row of the predictor varables will contain 4 consecutive rows of our original database, flattened out into [Month1, Tmin1, Tmax1, ..., Month4, Tmin4, Tmax4] which is exactly 12 values per row.
<p>Each row of the output variables will contain the lowest and highest temperatures of the following day: [Tmin5, Tmax5].
<p>This way our predictor has access to temperatures of the previous 4 days, and knows generally what season we're currently in.

In [ ]:
X = []
y = []
for i, row in weather.iterrows():
    if i < len(weather)-4:
        current_X_element = []
        for j in range(4):
            cur_row = weather[i+j:i+j+1]
            current_X_element.append(int(cur_row['month']))
            current_X_element.append(float(cur_row['Tmin']))
            current_X_element.append(float(cur_row['Tmax']))
        X.append(current_X_element)
        cur_target_row = weather[i+4:i+5]
        y.append([float(cur_target_row['Tmin']),
                  float(cur_target_row['Tmax'])])
X = np.array(X)
y = np.array(y)

The predictor variables look like the this:

In [ ]:
X[0:5]

array([[11. ,  5.3,  9.7, 11. ,  6.3, 11.1, 11. ,  9.8, 16.4, 11. ,  9.9,
        15.7],
       [11. ,  6.3, 11.1, 11. ,  9.8, 16.4, 11. ,  9.9, 15.7, 11. ,  9.2,
        16.3],
       [11. ,  9.8, 16.4, 11. ,  9.9, 15.7, 11. ,  9.2, 16.3, 11. ,  9.7,
        12.7],
       [11. ,  9.9, 15.7, 11. ,  9.2, 16.3, 11. ,  9.7, 12.7, 11. ,  9.6,
        13.9],
       [11. ,  9.2, 16.3, 11. ,  9.7, 12.7, 11. ,  9.6, 13.9, 11. ,  8.5,
        13.5]])

In [ ]:
X.shape

(1106, 12)

The output variables look like this:

In [ ]:
y[0:5]

array([[ 9.2, 16.3],
       [ 9.7, 12.7],
       [ 9.6, 13.9],
       [ 8.5, 13.5],
       [10.5, 15.1]])

In [ ]:
y.shape

(1106, 2)

Now we have the training data in such a form, that train-test-split can be used on it.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=random_state)
print("Shape of X_train: ", X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ", y_train.shape)
print("Shape of y_test: ", y_test.shape)

Shape of X_train:  (774, 12)
Shape of X_test:  (332, 12)
Shape of y_train:  (774, 2)
Shape of y_test:  (332, 2)


**Modelling**

The model has two hidden layers: Dense layers with 32 neurons, and relu activation. Then the last layers is a Dense with 2 neurons (it is going to predict the Tmin and Tmax values)

In [ ]:
model = Sequential()
model.add(Dense(units=32, input_dim=X.shape[1],kernel_regularizer=tf.keras.regularizers.l1(0.01), activation='relu'))
model.add(Dense(units=32, input_dim=32,kernel_regularizer=tf.keras.regularizers.l1(0.01), activation='relu'))
model.add(Dense(units=2))

model.compile(loss='mse', optimizer='adam', metrics=['acc'])

In [ ]:
es=EarlyStopping(patience=10, verbose=1)
cp=ModelCheckpoint(filepath='weights.hdf5', save_best_only=True, verbose=1)

In [ ]:
model.fit(X_train, y_train, batch_size=20, epochs=100, validation_split= 0.1, callbacks=[cp,es])

Epoch 1/100
26/35 [=====================>........] - ETA: 0s - loss: 194.8967 - acc: 0.8558 
Epoch 1: val_loss improved from inf to 36.52967, saving model to weights.hdf5
35/35 [==============================] - 1s 10ms/step - loss: 158.6073 - acc: 0.8649 - val_loss: 36.5297 - val_acc: 0.9231
Epoch 2/100
22/35 [=================>............] - ETA: 0s - loss: 18.9505 - acc: 0.9909
Epoch 2: val_loss improved from 36.52967 to 17.15287, saving model to weights.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 17.5601 - acc: 0.9943 - val_loss: 17.1529 - val_acc: 0.9872
Epoch 3/100
24/35 [===================>..........] - ETA: 0s - loss: 12.4042 - acc: 1.0000
Epoch 3: val_loss improved from 17.15287 to 14.50592, saving model to weights.hdf5
35/35 [==============================] - 0s 4ms/step - loss: 12.4128 - acc: 1.0000 - val_loss: 14.5059 - val_acc: 1.0000
Epoch 4/100
27/35 [======================>.......] - ETA: 0s - loss: 10.9339 - acc: 1.0000
Epoch 4: val_loss improve

Now let's see the prediction on X_test

In [ ]:
pred = model.predict(X_test)

11/11 [==============================] - 0s 2ms/step


We have to change the Tmin and Tmax values into an Avg value. The daily average temperature is the mean of Tmin and Tmax.

In [ ]:
pred_avg = np.array([np.mean(e) for e in pred])
actual_avg = np.array([np.mean(e) for e in y_test])

In [ ]:
test_err = mean_squared_error(actual_avg,pred_avg) #evaluating our model
test_err

3.993039660159962

This mse value shows, that the model is capable of predicting the next day's avg temperature with only a little mistake.



---





But in the exercise, the model has to predict not just the following day, but multiple days ahead. To do this I am going to use the following method:
<p>I am going to predict the following day's temperature and then add it to the known values of the model. Then repeat this.
<p>For this I'll need a fully ordered set of the days and temperatures, so I'll use the original X and y variables, and not X_train and y_train that only contain 70% of the days in our database. But the model stays the same.

The following function creates a continously updating list of the temperatures of the last 12 days (the last_12_days variable), calculates a prediction of the next day, appends it to this list, and deletes the first row of last_12_days. It does this until it reaches the day we're looking for.

In [ ]:
today = list(weather['date'])[-1]

def predict_temperatures(days_from_now):
    predictions = [] 
    day_needed = today + timedelta(days=days_from_now)
    last_12_days = X[-12:]
    for i in range(days_from_now):
        next_pred = model.predict(last_12_days,verbose=0)[-1]
        predictions.append(next_pred)
        last_12_days = last_12_days[1:]
        last_12_days = np.array(list(last_12_days) + list([np.append(np.append(last_12_days[-1][3:],[day_needed.month]),next_pred)]))
    return predictions

Here is an example: These are the predicted temperatures of the following 10 days:

In [ ]:
def spaces(l):
    return ''.join([' ' for k in range(l)])

next10_pred = predict_temperatures(10)
print("    Day   | Predicted minimum temperature | Predicted maximum temperature")
print("-------------------------------------------------------------------------")
for r in range(len(next10_pred)):
    day = today + timedelta(days=r+1)
    m = calendar.month_abbr[day.month]
    d = day.day
    mint = next10_pred[r][0]
    maxt = next10_pred[r][1]
    print(" ", m, d, spaces(5-len(str(m))-len(str(d))), "|", spaces(9), mint, spaces(18-len(str(mint))), "|", spaces(9), maxt)

    Day   | Predicted minimum temperature | Predicted maximum temperature
-------------------------------------------------------------------------
  Nov 15  |           8.521012            |           12.862491
  Nov 16  |           8.447227            |           12.798655
  Nov 17  |           8.313172            |           12.737107
  Nov 18  |           8.059712            |           12.381327
  Nov 19  |           7.869197            |           12.107246
  Nov 20  |           7.6978436           |           11.913091
  Nov 21  |           7.5161695           |           11.718425
  Nov 22  |           7.322652            |           11.477344
  Nov 23  |           7.1218276           |           11.226636
  Nov 24  |           6.9171867           |           10.97353


The following function predicts for a specific day:

In [ ]:
def predict_faraway_day_temperature(year, month, day):
    d = date(year, month, day)
    today = list(weather['date'])[-1]
    delta = (d - date(today.year, today.month, today.day)).days
    return predict_temperatures(delta)[-1]

Here are the actual predictions the exercise asked for:

In [ ]:
print("Predicted average temperatures in Budapest (°C):")
print("November 16th:", np.mean(predict_faraway_day_temperature(2022,11,16)))
print("November 22nd:", np.mean(predict_faraway_day_temperature(2022,11,22)))
print("December 6th:", np.mean(predict_faraway_day_temperature(2022,12,6)))

Predicted average temperatures in Budapest (°C):
November 16th: 10.62294
November 22nd: 9.399998
December 6th: 3.672347
